# 如何在测量过程中坍缩状态？
正如在如何执行测量？中所提到的，默认情况下，测量只能在电路的末尾使用，并且它们对状态没有任何影响。在本节中，我们将描述如何在测量过程中坍缩状态，并在电路中重新使用已测量的量子比特。状态坍缩意味着根据每个被测量量子比特的采样结果，将其投影到|0>或|1>子空间。

当在qibo.gates.M门的实例化过程中使用collapse=True时，状态会发生坍缩。例如

In [21]:
circuit.summary()

'Circuit depth = 4\nTotal number of gates = 5\nNumber of qubits = 2\nMost common gates:\nh: 3\nmeasure: 2'

In [23]:
import re

def get_total_gates(circuit_summary):
    """
    从电路摘要中提取总门数的函数
    
    参数:
        circuit_summary (str): circuit.summary()的输出字符串
        
    返回:
        int: 总门数
    """
    match = re.search(r'Total number of gates = (\d+)', circuit_summary)
    if match:
        return int(match.group(1))
    return None  # 如果没有找到匹配项，返回None

# 使用示例：
# text = circuit.summary()
# total_gates = get_total_gates(text)
# print(total_gates)


In [25]:
# 使用示例：
text = circuit.summary()
total_gates = get_total_gates(text)
type(total_gates)


int

In [1]:
from qibo import Circuit, gates

# 创建一个量子电路，包含1个量子比特，并启用密度矩阵模拟
circuit = Circuit(1, density_matrix=True)

# 在量子比特0上添加Hadamard门
circuit.add(gates.H(0))

# 对量子比特0进行测量，并在测量后进行状态坍缩
output = circuit.add(gates.M(0, collapse=True))

# 再次在量子比特0上添加Hadamard门
circuit.add(gates.H(0))

# 执行电路，进行1次测量
result = circuit(nshots=1)

# 输出测量结果，如果测量到0，则输出 |+><+|，如果测量到1，则输出 |-><-|
print(result)


[Qibo 0.2.21|INFO|2025-10-11 16:48:13]: Using qibojit (numba) backend on /CPU:0


(0.5+0j)|0><0| + (-0.5+0j)|0><1| + (-0.5+0j)|1><0| + (0.5+0j)|1><1|


In [13]:
result = circuit(nshots=1)
print(result)

(0.5+0j)|0><0| + (0.5+0j)|0><1| + (0.5+0j)|1><0| + (0.5+0j)|1><1|


在这个示例中，单个量子比特在处于状态（|0> + |1>）时被测量，并坍缩为|0>或|1>。之后，通过添加更多作用于该量子比特的门，它可以被重新使用。collapse=True测量的结果不包含在最终的结果对象中，但可以从向电路添加门时返回的output对象中获取。output支持output.samples()和output.frequencies()功能，如如何执行测量？

坍缩门默认是单次的，因为对于多次来说，状态坍缩的定义并不明确。如果用户在电路执行期间传递了nshots参数（例如，在上面的示例中使用result = c(nshots=100)），那么电路执行将通过循环重复nshots次：

In [15]:
for _ in range(100):
    result = circuit(nshots=1)

在同一个电路中同时使用常规测量和坍缩测量也是可行的：

In [16]:
from qibo import Circuit, gates

circuit = Circuit(2)
circuit.add(gates.H(0))
circuit.add(gates.H(1))
output = circuit.add(gates.M(0, collapse=True))
circuit.add(gates.H(0))
circuit.add(gates.M(0, 1))
result = circuit(nshots=100)

In [18]:
print(output.__dict__)

{'target_qubits': (0,), 'circuit': <qibo.models.circuit.Circuit object at 0x0000026BEFE83290>, '_samples': [array([0]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([1]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([1]), array([0]), array([0]), array([1]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([1]), array([1]), array([1]), array([0]), array([0]), array([1]), array([0]), array([1]), array([1]), array([0]), array([0]), array([0]), array([1]), array([0]), array([1]), arra